# Foundations of Computer Science, Project 2019
## Simone Tufano, 816984

## 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender

In [1]:
#importare le librerie e leggere il file loans_lenders.csv

import pandas as pd
import numpy as np
loans_lenders = pd.read_csv('loans_lenders.csv') 
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [160]:
#Poichè il file presenta oltre un milione di righe, andare a eseguire delle operazioni sul dataset intero causava
#'MemoryError' quindi, si decide di creare una lista di campioni e svolgere le azioni richieste per ogni porzione 
#di dataframe

loans_lenders_df_list = [loans_lenders.loc[0:100000]]
list_for_loc = [ 100000, 200000, 300000, 400000, 
                500000, 600000, 700000, 800000, 900000,
               1000000,1100000, 1200000, 1300000, 1387432]

for el in range(1,14):
    df = loans_lenders.loc[list_for_loc[el-1] + 1:list_for_loc[el]]
    loans_lenders_df_list.append(df)



In [162]:
#Check di verifica

loans_lenders_df_list[4].head()

,loan_id,lenders
400001,248833,"kim8007, tim5655, judy4594, mickey2663, harrie..."
400002,263054,"kim8007, romin3340, romin3340, romin3340, romi..."
400003,283527,kim8007
400004,308890,kim8007
400005,353961,"kim8007, ki3695, steve3540, christiane7688, ma..."


In [73]:
#Ciclo per normalizzare i dataset presenti in loans_lenders_df_list

new_loans_lenders_df_list  = []

for df in loans_lenders_df_list:
    new_loans_lenders_df = pd.DataFrame(df['lenders'].str.split(',').tolist(), 
                                     index = df['loan_id']).stack()
    new_loans_lenders_df = new_loans_lenders_df.reset_index([0,'loan_id'])
    new_loans_lenders_df.columns = ['loan_id','lenders']
    new_loans_lenders_df_list.append(new_loans_lenders_df)


In [78]:
#Check di verifica

new_loans_lenders_df_list[4]

,loan_id,lenders
0,248833,kim8007
1,248833,tim5655
2,248833,judy4594
3,248833,mickey2663
4,248833,harriet4014
5,248833,chris6099
6,248833,mark6087
7,248833,william6330
8,248833,kent6652
9,248833,seanIDS


In [80]:
#Concatenazione dei dataset presenti nella lista

new_loans_lenders_complete = pd.concat(new_loans_lenders_df_list)
new_loans_lenders_complete.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [81]:
#Poichè il file presenta delle dimensioni molto grandi e può servirmi per usi futuri, 
#viene salvato con la funzione to_pickle()

new_loans_lenders_complete.to_pickle('./dummy.pkl')

## 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [2]:
#Lettura del file loans.csv e controllo colonne

loans = pd.read_csv('loans.csv')
loans.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

In [86]:
#Check di verifica per la colonna 'planned_expiration_time'

loans['planned_expiration_time'].head()

0    2014-02-14 03:30:06.000 +0000
1    2014-03-26 22:25:07.000 +0000
2    2014-02-15 21:10:05.000 +0000
3    2014-02-21 03:10:02.000 +0000
4    2014-02-13 06:10:02.000 +0000
Name: planned_expiration_time, dtype: object

In [87]:
#Check di verifica per la colonna 'disburse_time'

loans['disburse_time'].head()

0    2013-12-22 08:00:00.000 +0000
1    2013-12-20 08:00:00.000 +0000
2    2014-01-09 08:00:00.000 +0000
3    2014-01-17 08:00:00.000 +0000
4    2013-12-17 08:00:00.000 +0000
Name: disburse_time, dtype: object

In [50]:
#Creazione di una nuova colonna 'duration' ottenuta dalla differenza delle due colonne precedenti convertite
#attraverso la funzione to_datetime()

loans[['planned_expiration_time','disburse_time']] = loans[['planned_expiration_time','disburse_time']].apply(pd.to_datetime)
loans['duration'] = (loans['planned_expiration_time'] - loans['disburse_time']).dt.days
loans['duration'].head()

0    53.0
1    96.0
2    37.0
3    34.0
4    57.0
Name: duration, dtype: float64

In [89]:
#Check di verifica per controllare se le osservazioni relative alla variabile 'duration' fossero nulle per osservazioni
#mancanti delle colonne 'planned_expiration_time' e 'disburse_time'

loans[loans['planned_expiration_time'].isna()].head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
64,251406,José Roan Group,Spanish,El grupo se llama “José Roan” y se ubica en l...,The group called “José Roan” is located in the...,825.0,825.0,funded,Poultry,Agriculture,...,7.0,22,2,1,NaN,"female, female, female","true, true, true",monthly,field_partner,NaN
65,252537,Marylou,English,"Marylou S. is from the village of Kuguita, Mam...",NaN,175.0,175.0,funded,Personal Products Sales,Retail,...,7.0,7,2,1,NaN,female,true,irregular,field_partner,NaN
66,253044,Flora,English,"Flora G. is from the village of Libona, Bukidn...",NaN,125.0,125.0,funded,Transportation,Transportation,...,7.0,5,2,1,NaN,female,true,irregular,field_partner,NaN
67,254012,Maria Magdalena,Spanish,"María de 55 años, tiene una tienda de víveres ...","María, 55, has a grocery store in the front pa...",1000.0,1000.0,funded,General Store,Retail,...,12.0,36,1,1,NaN,female,true,monthly,field_partner,NaN
69,251336,Rober,Spanish,"Rober, de 32 años, está casado con Janeth, de ...","Rober, 32, is married to Janeth, 31, and they ...",375.0,375.0,funded,Knitting,Arts,...,12.0,15,3,2,NaN,male,true,monthly,field_partner,NaN


In [90]:
loans[loans['disburse_time'].isna()].head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
31106,1379508,Melissa,English,I grew up in Chicago and started playing viola...,I grew up in Chicago and started playing viola...,125.0,1000.0,expired,Education provider,Education,...,12.0,2,1,1,NaN,female,true,bullet,direct,NaN
31107,1428617,Dwarika,English,It all started when I was not conscious about ...,It all started when I was not conscious about ...,0.0,10000.0,fundRaising,Technology,Services,...,36.0,0,1,1,NaN,male,true,bullet,direct,NaN
31178,1135319,Robert,English,"My name is Roberto Hernandez, I grew up in a s...","My name is Roberto Hernandez, I grew up in a s...",50.0,500.0,expired,Transportation,Transportation,...,24.0,2,1,1,NaN,male,true,bullet,direct,NaN
32151,1279252,Michal,English,My name is Michal Szymczak and I am a Polish i...,My name is Michal Szymczak and I am a Polish i...,0.0,1000.0,expired,Services,Services,...,12.0,0,1,1,NaN,male,true,bullet,direct,NaN
32152,1433349,NaN,NaN,NaN,NaN,0.0,8000.0,refunded,Services,Services,...,36.0,0,1,1,NaN,NaN,NaN,bullet,direct,NaN


## 3. Find the lenders that have funded at least twice.

In [33]:
#Lettura del file lenders.csv per controllo (non necessaria per lo svolgimento del punto)

lenders = pd.read_csv('lenders.csv')
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [82]:
#Controllo del DataFrame ottenuto dal file loans_lenders.csv del punto 1

new_loans_lenders_complete.head() 

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [83]:
#DataFrame con lenders duplicati, ottenuto attraverso la funzione .duplicated()

new_loans_lenders_complete[new_loans_lenders_complete.duplicated('lenders')]

,loan_id,lenders
40,483738,muc888
45,483738,danhostetler
46,483738,danhostetler
55,485000,muc888
59,485000,mattiaslaven
60,485000,shree8053
62,485000,don9212
63,485000,raph8817
66,485000,highgrovechurch
67,485000,dino5102


## 4. For each country, compute how many loans have involved that country as borrowers.

In [92]:
#Controllo sui nomi dei paesi

loans.country_name.head()

0    Philippines
1       Honduras
2       Pakistan
3     Kyrgyzstan
4    Philippines
Name: country_name, dtype: object

In [93]:
#Check di verifica per controllare se i codici dei paesi e i loro nomi potessero essere utilizzati come variabile di
#raggruppamento

print(len(loans['country_name']), len(loans['country_code']))

1419607 1419607


In [94]:
#Raggruppamento attraverso .groupby() e conteggio delle osservazioni (numero di loans) con la funzione .size()

loans.groupby('country_name').size()

country_name
Afghanistan                               2337
Albania                                   3075
Armenia                                  13952
Azerbaijan                               10172
Belize                                     218
Benin                                     5946
Bhutan                                       2
Bolivia                                  25250
Bosnia and Herzegovina                     608
Botswana                                     1
Brazil                                     482
Bulgaria                                   296
Burkina Faso                              3489
Burundi                                   1727
Cambodia                                 79701
Cameroon                                  5085
Canada                                       1
Chad                                        61
Chile                                      892
China                                      134
Colombia                                 33675


## 5. For each country, compute the overall amount of money borrowed.

In [95]:
#Raggruppamento come nel punto precedente e somma della variabile 'loan_amount' attraverso la funzione .sum()

loans.groupby('country_name')['loan_amount'].sum() 

country_name
Afghanistan                              1967950.0
Albania                                  4307350.0
Armenia                                 22950475.0
Azerbaijan                              14784625.0
Belize                                    150175.0
Benin                                    3865825.0
Bhutan                                     20000.0
Bolivia                                 44226725.0
Bosnia and Herzegovina                    477250.0
Botswana                                    8000.0
Brazil                                   1192325.0
Bulgaria                                  375300.0
Burkina Faso                             4085200.0
Burundi                                  5233450.0
Cambodia                                51613525.0
Cameroon                                 2250800.0
Canada                                     50000.0
Chad                                       20075.0
Chile                                    2595925.0
China             

## 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [96]:
#Totale dei soldi prestati

overall_amount_lent = loans['loan_amount'].sum()
overall_amount_lent

1181437300.0

In [97]:
#Raggruppamento come i due punti precedenti e divisione per il totale dei soldi prestati

loans.groupby('country_name')['loan_amount'].sum()/overall_amount_lent

country_name
Afghanistan                             0.001666
Albania                                 0.003646
Armenia                                 0.019426
Azerbaijan                              0.012514
Belize                                  0.000127
Benin                                   0.003272
Bhutan                                  0.000017
Bolivia                                 0.037435
Bosnia and Herzegovina                  0.000404
Botswana                                0.000007
Brazil                                  0.001009
Bulgaria                                0.000318
Burkina Faso                            0.003458
Burundi                                 0.004430
Cambodia                                0.043687
Cameroon                                0.001905
Canada                                  0.000042
Chad                                    0.000017
Chile                                   0.002197
China                                   0.000322
Colombi

## 7. Like the three previous points, but split for each year (with respect to disburse time).

In [98]:
#Creazione di una variabile 'year' per il raggruppamento e un nuovo dataframe. 

loans['year'] = loans['disburse_time'].dt.year
loans_df_groupby = pd.DataFrame({})
loans_df_groupby['number_of_loans'] =  loans.groupby(['year','country_name'])['loan_amount'].size()
loans_df_groupby['loan_amount'] =  loans.groupby(['year','country_name'])['loan_amount'].sum()
loans_df_groupby['perc_of_overall_amount_lent'] =  loans.groupby(['year','country_name'])['loan_amount'].sum()/overall_amount_lent
loans_df_groupby

number_of_loans  loan_amount  \
year   country_name                                                     
2005.0 Bulgaria                                        3       1850.0   
       Cambodia                                        2       1625.0   
       Ecuador                                        10       5000.0   
       Gaza                                            8       5000.0   
       Honduras                                       72      33300.0   
       Kenya                                          40      21350.0   
       Nicaragua                                       6       3300.0   
       Senegal                                         3       3225.0   
       Tanzania                                        5       2300.0   
       Uganda                                         54      25900.0   
2006.0 Azerbaijan                                     21      15700.0   
       Bulgaria                                       70      84650.0   
       Cambodia                                       58      41950.0   
       Ecuador                                       253     151175.0   
       Ghana                                          28      23425.0   
       Honduras                                      272      94325.0   
       Kenya                                         434     222800.0   
       Mexico                                        285     191575.0   
       Moldova                                        31      20925.0   
       Mozambique                                     16      15950.0   
       Nicaragua                                       5       3850.0   
       Nigeria                                       102      46950.0   
       Samoa                                          13       7800.0   
       Senegal                                        54      52225.0   
       Tanzania                                       69      45200.0   
       Togo                                           95      77600.0   
       Uganda                                        366     280475.0   
2007.0 Afghanistan                                   408     194975.0   
       Azerbaijan                                   2133    1892775.0   
       Bolivia                                       565     304700.0   
...                                                  ...          ...   
2018.0 Costa Rica                                      1       1550.0   
       Dominican Republic                              2       9850.0   
       Ghana                                          67      74675.0   
       Guatemala                                       1       3500.0   
       Haiti                                          17      32950.0   
       India                                           1      20000.0   
       Indonesia                                       3       3450.0   
       Kenya                                         587     393350.0   
       Kyrgyzstan                                      1       8000.0   
       Lao People's Democratic Republic               19       6075.0   
       Malawi                                          2       2100.0   
       Mali                                            1      50000.0   
       Mexico                                          5        475.0   
       Moldova                                         5      15675.0   
       Mozambique                                     26      53850.0   
       Nicaragua                                       7      13425.0   
       Nigeria                                        10      13900.0   
       Pakistan                                        6        150.0   
       Panama                                          1      30000.0   
       Paraguay                                        2         50.0   
       Peru                                           10      17700.0   
       Philippines                                     6        300.0   
       Rwanda             

## 8.For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [9]:
#Creazione di una nuova variabile con la lunghezza della lista dei lenders

loans_lenders['len_of_list'] = loans_lenders['lenders'].str.split(',')
loans_lenders['len_of_list'] = loans_lenders['len_of_list'].apply(lambda x: len(x))
loans_lenders.head()

,loan_id,lenders,len_of_list
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb...",40
1,483738,"muc888, nora3555, williammanashi, barbara5610,...",15
2,485000,"muc888, terrystl, richardandsusan8352, sherri4...",15
3,486087,"muc888, james5068, rudi5955, daniel9859, don92...",13
4,534428,"muc888, niki3008, teresa9174, mike4896, david7...",19


In [16]:
#Recupero file punto 1

new_loans_lenders_complete = pd.read_pickle('./dummy.pkl')
new_loans_lenders_complete.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [17]:
#Merge con file 'loans_lenders' e 'new_loans_lenders_complete' per ottenere il dataset normalizzato con la variabile
#relativa la lunghezza della lista a cui appartiene il lender

new_loans_lenders_complete = new_loans_lenders_complete.merge(loans_lenders[['len_of_list','loan_id']])
new_loans_lenders_complete.head()

,loan_id,lenders,len_of_list
0,483693,muc888,40
1,483693,sam4326,40
2,483693,camaran3922,40
3,483693,lachheb1865,40
4,483693,rebecca3499,40


In [18]:
#Selezione delle variabili di interesse

loans_min = loans[['loan_id','loan_amount']]
loans_min.head()

,loan_id,loan_amount
0,657307,125.0
1,657259,400.0
2,658010,400.0
3,659347,625.0
4,656933,425.0


In [19]:
#Nuovo merge

loans_min_merge = loans_min.merge(new_loans_lenders_complete)
loans_min_merge.head()

,loan_id,loan_amount,lenders,len_of_list
0,657307,125.0,spencer5657,3
1,657307,125.0,matthew8640,3
2,657307,125.0,larry71496105,3
3,657259,400.0,ltr,7
4,657259,400.0,andrew5306,7


In [20]:
#Divisione dell'importo 'loan_amount' per il numero di lenders che hanno contriuito al prestito

loans_min_merge['real_loan_amount'] = loans_min_merge['loan_amount'] / loans_min_merge['len_of_list']
loans_min_merge.head()

,loan_id,loan_amount,lenders,len_of_list,real_loan_amount
0,657307,125.0,spencer5657,3,41.666667
1,657307,125.0,matthew8640,3,41.666667
2,657307,125.0,larry71496105,3,41.666667
3,657259,400.0,ltr,7,57.142857
4,657259,400.0,andrew5306,7,57.142857


In [57]:
money_lent = pd.DataFrame(loans_min_merge.groupby('lenders')['real_loan_amount'].sum()).reset_index().round()
money_lent.head()

,lenders,real_loan_amount
0,000,1673.0
1,00000,1381.0
2,0002,2473.0
3,00mike00,53.0
4,0101craign0101,2624.0


## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [40]:
#Controllo di lenders.csv

lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
5,sirinapa6764,Sirinapa,NaN,NaN,NaN,1461301496,NaN,NaN,1.0,NaN,0
6,rene7585,Rene,NaN,NaN,NaN,1461301636,NaN,NaN,2.0,NaN,0
7,harald2826,Harald,NaN,NaN,NaN,1461301670,NaN,NaN,2.0,NaN,0
8,mehdi2903,Mehdi,NaN,NaN,NaN,1461301756,NaN,NaN,NaN,NaN,0
9,youchan8125,Youchan,NaN,NaN,NaN,1461301941,NaN,NaN,1.0,NaN,0


In [23]:
#Creazione di un DataFrame con i soldi prestati da ogni paese

money_borrowed = pd.DataFrame(loans.groupby('country_code')['loan_amount'].sum())
money_borrowed = money_borrowed.rename(columns={'loan_amount':'money_borrowed'})
money_borrowed.reset_index()

,country_code,money_borrowed
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
5,BF,4085200.0
6,BG,375300.0
7,BI,5233450.0
8,BJ,3865825.0
9,BO,44226725.0


In [58]:
money_lent = money_lent.rename(columns={'real_loan_amount':'money_lent'})
money_lent.head()

,lenders,money_lent
0,000,1673.0
1,00000,1381.0
2,0002,2473.0
3,00mike00,53.0
4,0101craign0101,2624.0


In [34]:
#Creazione del dataset con le variabili per il merge e rimozione dei valori mancanti

lenders_for_merge = pd.DataFrame(lenders[['country_code','permanent_name']])
lenders_for_merge = lenders_for_merge.dropna() 
lenders_for_merge

,country_code,permanent_name
16,US,naresh2074
31,US,christina27976796
37,IN,vikas1098
39,US,qian1385
42,US,xigg8769
45,US,prathapkiran5174
54,US,prashankpradeep8456
66,US,sethumathavan2571
77,AU,jack8790
91,US,abpestcontrol


In [59]:
lenders_for_merge_1 = lenders_for_merge.merge(money_lent, left_on = 'permanent_name', right_on = 'lenders')
lenders_for_merge_1

,country_code,permanent_name,lenders,money_lent
0,US,naresh2074,naresh2074,64.0
1,US,prashankpradeep8456,prashankpradeep8456,30.0
2,FR,pierremarie5489,pierremarie5489,34.0
3,SK,radoslava6314,radoslava6314,38.0
4,US,sinsin2875,sinsin2875,33.0
5,US,jasmine36996080,jasmine36996080,208.0
6,US,alaynna2764,alaynna2764,720.0
7,GB,engin6513,engin6513,41.0
8,US,davewheeler,davewheeler,358.0
9,US,muge3368,muge3368,29.0


In [60]:
#Calcolo delle differenze tra soldi prestati e ricevuti per ogni paese

country_difference = lenders_for_merge_1.merge(money_borrowed, on='country_code')
country_difference['difference'] = country_difference['money_borrowed'] - country_difference['money_lent']
diz_cd = country_difference.groupby('country_code')['difference'].sum().to_dict()
diz_cd

{'AF': 61000188.0,
 'AL': 34458068.0,
 'AM': 137701806.0,
 'AZ': 59138317.0,
 'BA': 5721309.0,
 'BG': 22874731.0,
 'BJ': 3865692.0,
 'BO': 442265544.0,
 'BR': 538858728.0,
 'BT': 19941.0,
 'BZ': 750534.0,
 'CA': 898319118.0,
 'CG': 2265206.0,
 'CL': 150551929.0,
 'CM': 4501540.0,
 'CN': 74157701.0,
 'CO': 1676572352.0,
 'CR': 694233819.0,
 'DO': 121002953.0,
 'EC': 994517140.0,
 'EG': 66498389.0,
 'GE': 38281524.0,
 'GH': 183148363.0,
 'GT': 374351405.0,
 'GU': 203452.0,
 'HN': 203816942.0,
 'HT': 8992159.0,
 'ID': 818541812.0,
 'IL': 417534696.0,
 'IN': 3694105752.0,
 'IQ': 91660489.0,
 'JO': 231152731.0,
 'KE': 1935336973.0,
 'KG': 88359037.0,
 'KH': 1238720381.0,
 'LA': 161873445.0,
 'LB': 301367091.0,
 'LK': 746690.0,
 'LR': 7857214.0,
 'LS': 516337.0,
 'MD': 2714498.0,
 'MG': 1538988.0,
 'ML': 15504617.0,
 'MM': 8402714.0,
 'MN': 168827953.0,
 'MW': 8729915.0,
 'MX': 7531830979.0,
 'MZ': 12542928.0,
 'NG': 108523043.0,
 'NI': 361835507.0,
 'NP': 6048601.0,
 'PA': 11238533.0,
 'PE'

In [61]:
#DataFrame con le differenze per ogni paese

difference_db = pd.DataFrame(list(diz_cd.items()),
             columns = ['country_code','difference_OML_OMB'])
difference_db.head()

,country_code,difference_OML_OMB
0,AF,61000188.0
1,AL,34458068.0
2,AM,137701806.0
3,AZ,59138317.0
4,BA,5721309.0


## 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [41]:
#Lettura del file country_stats.csv

country_stats = pd.read_csv('country_stats.csv')
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
5,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo
6,United States,US,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.538150,13.217890,53245.076960,United States
7,China,CN,CHN,Asia,Eastern Asia,1409517397,3.3,0.737681,75.963,13.535750,7.641840,13345.477460,China
8,Ethiopia,ET,ETH,Africa,Eastern Africa,104957438,29.6,0.447750,64.602,8.351520,2.580780,1522.954782,Ethiopia
9,Indonesia,ID,IDN,Asia,South-eastern Asia,263991379,10.9,0.688847,69.052,12.867660,7.934817,10053.337700,Indonesia


In [62]:
#Merge con il DataFrame ottenuto al punto precedente 

highest_ratio = difference_db.merge(country_stats[['country_code', 'population']])
highest_ratio.head()

,country_code,difference_OML_OMB,population
0,AF,61000188.0,35530081
1,AL,34458068.0,2930187
2,AM,137701806.0,2930450
3,AZ,59138317.0,9827589
4,BA,5721309.0,3507017


In [63]:
#Creazione di una variabile 'ratio' con il rapporto fra la differenza dei soldi prestati e ricevuti e la popolazione

highest_ratio['ratio'] = highest_ratio['difference_OML_OMB']/highest_ratio['population']
highest_ratio.head()

,country_code,difference_OML_OMB,population,ratio
0,AF,61000188.0,35530081,1.716860
1,AL,34458068.0,2930187,11.759682
2,AM,137701806.0,2930450,46.989987
3,AZ,59138317.0,9827589,6.017581
4,BA,5721309.0,3507017,1.631389


In [66]:
#Ordinamento in base al valore del rapporto più alto

highest_ratio.sort_values('ratio', ascending = False)

,country_code,difference_OML_OMB,population,ratio
72,US,5.236486e+12,324459463,16139.105705
53,PH,2.616182e+10,104918090,249.354666
17,CR,6.942338e+08,4905769,141.513760
33,KH,1.238720e+09,16005373,77.394034
51,PE,2.383126e+09,32165485,74.089535
63,SV,4.586049e+08,6377853,71.905839
19,EC,9.945171e+08,16624858,59.821091
45,MX,7.531831e+09,129163276,58.312480
48,NI,3.618355e+08,6217581,58.195544
43,MN,1.688280e+08,3075647,54.891850


## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [67]:
#Merge con il DataFrame precedente e la colonna 'population_below_poverty_line' di country_stats

highest_ratio = pd.merge(highest_ratio, country_stats[['country_code','population_below_poverty_line']], 
                         on = 'country_code', how = 'left')
highest_ratio.head()

,country_code,difference_OML_OMB,population,ratio,population_below_poverty_line
0,AF,61000188.0,35530081,1.716860,35.8
1,AL,34458068.0,2930187,11.759682,14.3
2,AM,137701806.0,2930450,46.989987,32.0
3,AZ,59138317.0,9827589,6.017581,4.9
4,BA,5721309.0,3507017,1.631389,17.2


In [68]:
#La colonna 'population_below_poverty_line' sembra essere in valori percentuali, quindi viene ritrasformata in 
#valori assoluti

highest_ratio['real_pop_BPL'] = highest_ratio['population']*highest_ratio['population_below_poverty_line']//100
highest_ratio.head()

,country_code,difference_OML_OMB,population,ratio,population_below_poverty_line,real_pop_BPL
0,AF,61000188.0,35530081,1.716860,35.8,12719768.0
1,AL,34458068.0,2930187,11.759682,14.3,419016.0
2,AM,137701806.0,2930450,46.989987,32.0,937744.0
3,AZ,59138317.0,9827589,6.017581,4.9,481551.0
4,BA,5721309.0,3507017,1.631389,17.2,603206.0


In [69]:
#Calcolo del nuovo rapporto

highest_ratio['ratio_2'] = highest_ratio['difference_OML_OMB']/highest_ratio['real_pop_BPL']
highest_ratio.head()

,country_code,difference_OML_OMB,population,ratio,population_below_poverty_line,real_pop_BPL,ratio_2
0,AF,61000188.0,35530081,1.716860,35.8,12719768.0,4.795700
1,AL,34458068.0,2930187,11.759682,14.3,419016.0,82.235686
2,AM,137701806.0,2930450,46.989987,32.0,937744.0,146.843708
3,AZ,59138317.0,9827589,6.017581,4.9,481551.0,122.808004
4,BA,5721309.0,3507017,1.631389,17.2,603206.0,9.484834


In [70]:
#Ordinamento del nuovo rapporto

highest_ratio.sort_values('ratio_2', ascending = False)

,country_code,difference_OML_OMB,population,ratio,population_below_poverty_line,real_pop_BPL,ratio_2
72,US,5.236486e+12,324459463,16139.105705,15.1,48993378.0,106881.496728
53,PH,2.616182e+10,104918090,249.354666,21.6,22662307.0,1154.419770
17,CR,6.942338e+08,4905769,141.513760,21.7,1064551.0,652.137680
33,KH,1.238720e+09,16005373,77.394034,17.7,2832951.0,437.254432
51,PE,2.383126e+09,32165485,74.089535,22.7,7301565.0,326.385621
11,CA,8.983191e+08,36624199,24.528021,9.4,3442674.0,260.936446
43,MN,1.688280e+08,3075647,54.891850,21.6,664339.0,254.129222
19,EC,9.945171e+08,16624858,59.821091,25.6,4255963.0,233.676172
27,IL,4.175347e+08,8321570,50.174991,22.0,1830745.0,228.068189
63,SV,4.586049e+08,6377853,71.905839,34.9,2225870.0,206.033987


## 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [51]:
#Creazione di un nuovo DataFrame con le sole variabili di interesse e creazione di un DataFrame per i test

loans_12 = loans[['loan_id','loan_amount','disburse_time','planned_expiration_time','duration']]
loans_12_tent = loans_12.loc[0:5]
loans_12_tent.head()

,loan_id,loan_amount,disburse_time,planned_expiration_time,duration
0,657307,125.0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53.0
1,657259,400.0,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96.0
2,658010,400.0,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37.0
3,659347,625.0,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34.0
4,656933,425.0,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57.0


In [52]:
#Non vengono prese in considerazione le osservazioni che hanno un valore di disburse_time' maggiore di 
#'planned_expiration_time'

loans_12 = loans_12[loans_12['duration'] >= 0]
loans_12.head()

,loan_id,loan_amount,disburse_time,planned_expiration_time,duration
0,657307,125.0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53.0
1,657259,400.0,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96.0
2,658010,400.0,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37.0
3,659347,625.0,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34.0
4,656933,425.0,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57.0


In [53]:
#Check di verifica sulla lunghezza del file

len(loans_12)

1030027

In [54]:
#Check di verifica sulla funzione .dayofyear

pd.Period(str(loans_12['disburse_time'][1])).dayofyear

354

In [55]:
#Creazione di una funzione in grado di ritornare una lista di liste, con input due date, costituita
#da anno/i in considerazione con i rispettivi pesi (giorni)

def list_year_date(date_dis, date_et):
    diff = pd.to_datetime(date_et).year - pd.to_datetime(date_dis).year #numero di elementi per cui deve essere ponderato il loan_amount
    lista_temp = list(range(0,int(diff+1)))
    if len(lista_temp) == 1:
        lista_temp[0] = [pd.to_datetime(date_et).year, pd.Period(str(date_et)).dayofyear - pd.Period(str(date_dis)).dayofyear]
    if len(lista_temp) == 2:                     
        diff_1 = (365 - pd.Period(str(date_dis)).dayofyear)
        diff_2 = pd.Period(str(date_et)).dayofyear
        lista_temp[0] = [pd.to_datetime(date_dis).year, diff_1]
        lista_temp[1] = [pd.to_datetime(date_et).year, diff_2]
    if len(lista_temp) > 2:
        diff_1 = (365 - pd.Period(str(date_dis)).dayofyear)
        diff_2 = pd.Period(str(date_et)).dayofyear
        lista_temp[0] = [pd.to_datetime(date_dis).year, diff_1]
        lista_temp[-1] = [pd.to_datetime(date_et).year, diff_2]
        a = pd.to_datetime(date_dis).year + 1
        for yr in range(1, len(lista_temp)-1):
            lista_temp[yr] = [a, 365]
            a += 1
    return lista_temp

                         
l = list_year_date('2016-12-1','2018-1-30')
l     


[[2016, 29], [2017, 365], [2018, 30]]

In [56]:
#Creazione di una funzione che ritornasse un dizionario di liste, con i valori di loan_amount già ponderati per ogni anno
#per ogni osservazione (loan_id)

def list_partial_loan(lista_date, loan_amount, loan_id):
    diz_partial_loan = {}
    lista = []
    somma_giorni = 1
    for year in lista_date:
        somma_giorni += year[1]
    if somma_giorni > 0:
        for year in lista_date:
            lista.append([year[0], loan_amount*year[1]/somma_giorni])
    
    diz_partial_loan[loan_id] = lista
    return diz_partial_loan

                                   
diz = list_partial_loan(l, 5000, 1)
print(diz)
        

{1: [[2016, 341.1764705882353], [2017, 4294.117647058823], [2018, 352.94117647058823]]}


In [78]:
#Test su DataFrame ridotto

lista = []
for index, row in loans_12_tent.iterrows():
    l = list_year_date(row['disburse_time'], row['planned_expiration_time'])
    diz = list_partial_loan(l, row['loan_amount'], row['loan_id'])
    lista.append(diz)

loans_12_tent['diz'] = lista
loans_12_tent.head()

,loan_id,loan_amount,disburse_time,planned_expiration_time,duration,diz
0,657307,125.0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53.0,"{657307: [[2013, 20.454545454545453], [2014, 1..."
1,657259,400.0,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96.0,"{657259: [[2013, 45.36082474226804], [2014, 35..."
2,658010,400.0,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37.0,"{658010: [[2014, 389.4736842105263]]}"
3,659347,625.0,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34.0,"{659347: [[2014, 607.6388888888889]]}"
4,656933,425.0,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57.0,"{656933: [[2013, 100.84745762711864], [2014, 3..."


In [80]:
#Rimozione dati mancanti e check sulla lunghezza

loans_12 = loans_12.dropna()
len(loans_12)

1030027

In [81]:
#Applicazioni delle funzioni su loans_12 e salvataggio del file .pkl

list_for_new_variable = []


for index, row in loans_12.iterrows():
    l = list_year_date(row['disburse_time'], row['planned_expiration_time'])
    diz = list_partial_loan(l, row['loan_amount'], row['loan_id'])
    list_for_new_variable.append(diz)

loans_12['list_of_diz'] = list_for_new_variable
loans_12.to_pickle('./loans_123.pkl')


In [ ]:
#Lettura del file e check nuovo DataFrame

loans_12 = pd.read_pickle('./loans_123.pkl')
loans_12.head()

In [ ]:
#Alcuni indici sono mancanti, quindi è necessario un .reset_index() per non ottenere errori nel ciclo successivo

loans_12 = loans_12.reset_index()

In [123]:
#Creazione di un DataFrame in cui una o più osservazioni sono inerenti ad ogni prestito, con anno e importo ponderato

a = 0
lista_df = []

for i in loans_12['list_of_diz']:
    df = pd.DataFrame(pd.DataFrame(loans_12['list_of_diz'][a].values()).stack().tolist())
    a += 1
    lista_df.append(df)

,0,1
0,2013,20.454545
1,2014,102.272727
0,2013,45.360825
1,2014,350.515464
0,2014,389.473684


In [146]:
#Rinominazione delle colonne e creazione di un nuovo dataset con nuovi indici progressivi

loans_12_gb.columns = ['year','loan']
loans_12_new = loans_12_gb.reset_index(drop=True)
loans_12_new.head()

,year,loan
0,2013,20.454545
1,2014,102.272727
2,2013,45.360825
3,2014,350.515464
4,2014,389.473684


In [158]:
#Raggruppamento per ogni anno e somma degli importi ponderati, inoltre la funzione astype('int64') permette di
#non visualizzare l'importo complessivo in notazione scientifica

loans_12_new.groupby('year')['loan'].sum().astype('int64')

year
2011       620643
2012    110517808
2013    121013986
2014    147110340
2015    151384329
2016    149755340
2017    161346261
2018      4947277
Name: loan, dtype: int64